In [54]:
import sys
import os
sys.path.insert(0, os.path.abspath('../..'))
from scipy.io import loadmat
from pathlib import Path
import matplotlib.patches as patches
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
# the following import is required for matplotlib < 3.2:
from mpl_toolkits.mplot3d import Axes3D  # noqa
from scipy import stats
from scipy.stats import mannwhitneyu, ttest_rel
from scipy.signal import butter, filtfilt
import mne
import eeg_analysis.funcs4eeg as fe
import re
import imp
import ast
import behavior.func4behav as fb
imp.reload(fe)
imp.reload(fb)

from scipy import signal
from scipy.signal import resample
from scipy.ndimage import zoom

In [55]:
case_title = 'endo'
case_list = [case_title+' fast', case_title+' slow']
if 'endo' in case_title:
    tmax_list = [1.5+1+0.5+0.05+1, 1.5+1+1+0.05+1]
else:
    tmax_list = [1.5+0.033*4+0.5+0.05+1, 1.5+0.033*4+1+0.05+1]

fs = 1200
for i, case in enumerate(case_list):
    tmax = tmax_list[i]
    watch = '1 fixation'
    tmin = 0 # include fix or not?

    highpass = None # 0.3
    lowpass = None

    EP_lists, RT_lists = fe.pipeline_session_RT(case, watch, tmin, tmax, hipass=highpass, lopass=lowpass, baseline=(0,0), move_baseline=False, detrend=1)

    if 'slow' in case:
        sessions_slow = EP_lists
        rts_slow = RT_lists
    else:
        sessions_fast = EP_lists
        rts_fast = RT_lists


In [56]:
eegs_list = [sessions_fast, sessions_slow]
rts_list = [rts_fast, rts_slow]
session_names = ["sham before", "sham after", "real before", "real after"]
freq_band_names = ["theta", "alpha", "beta", "gamma"]
region_names = ["frontal", "central", "parietal", "occipital", "temporal", "all"]
stage_names = ["fixation", "cue", "wait", "response", "until response"]
freq_bands = [[4, 8], [8, 12], [12, 30], [30, 50]]
endo_fast_stages = [[0, 1.5], [1.5, 1.5+1], [1.5+1, 1.5+1+0.5], [1.5+1+0.5+0.05, 1.5+1+0.5+0.05+1], [0, 1.5+1+0.5+0.05]]
endo_slow_stages = [[0, 1.5], [1.5, 1.5+1], [1.5+1, 1.5+1+1], [1.5+1+1+0.05, 1.5+1+1+0.05+1], [0, 1.5+1+1+0.05+1]]
exo_fast_stages = [[0, 1.5], [1.5, 1.5+0.033*10], [1.5+0.033*4, 1.5+0.033*4+0.5], [1.5+0.033*4+0.5+0.05, 1.5+0.033*4+0.5+0.05+1], [0, 1.5+0.033*4+0.5+0.05]]
exo_slow_stages = [[0, 1.5], [1.5, 1.5+0.033*10], [1.5+0.033*4, 1.5+0.033*4+1], [1.5+0.033*4+1+0.05, 1.5+0.033*4+1+0.05+1], [0, 1.5+0.033*4+1+0.05]]
if 'endo' in case_title:
    stages_list = [endo_fast_stages, endo_slow_stages]
else:
    stages_list = [exo_fast_stages, exo_slow_stages]
bp_rt = pd.DataFrame(columns=['stage', 'band', 'region', 'session', 'group_id', 'band power', 'reaction time'])

In [57]:
eegs_list[0][0][1].shape

(28, 32, 4861)

In [58]:
for i, sessions in enumerate(eegs_list):
    rts = rts_list[i]
    stage_list = stages_list[i]
    for j, session_name in enumerate(session_names):
        session_eeg = sessions[j]
        session_rt = rts[j]
        # print(session_name)
        for group_id in range(9):
            # print(group_id)
            eeg = session_eeg[group_id]
            rt = session_rt[group_id]
            for trial in range(eeg.shape[0]):
                trial_data = eeg[trial,:,:]
                for k, stage in enumerate(stage_list):
                    stage_start = int(stage[0]*fs)
                    stage_end = int(stage[1]*fs)
                    psds, freqs = mne.time_frequency.psd_array_multitaper(trial_data[:,stage_start:stage_end], fs, fmin=1, fmax=51, verbose=False)
                    for l, freq_band in enumerate(freq_bands):
                        bp = fe.band_power(psds, freqs, freq_band)
                        for region in region_names:
                            channels_dict = fe.pick_cortex(region)
                            channel_names_list = list(channels_dict.keys())
                            channels = [channels_dict[key]-1 for key in channel_names_list]
                            bp_region = np.nanmean(bp[channels])
                            new_row = pd.DataFrame({
                                'stage': [stage_names[k]],
                                'band': [freq_band_names[l]],
                                'region': [region],
                                'session': [session_name],
                                'group_id': [group_id],
                                'band power': [bp_region],
                                'reaction time': [rt[trial]]
                            })
                            bp_rt = pd.concat([bp_rt, new_row], ignore_index=True)
bp_rt
# 11min32s

In [66]:
bp_rt_session = pd.DataFrame(columns=['stage', 'band', 'region', 'band power', 'reaction time'])
for stage in stage_names:
    for band in freq_band_names:
        for region in region_names:
            for session in session_names:
                for group_id in range (9):
                    bp_mean = bp_rt[(bp_rt['stage']==stage) & (bp_rt['band']==band) & (bp_rt['region']==region) & (bp_rt['session']==session) & (bp_rt['group_id']==group_id)]['band power'].mean()
                    rt_mean = bp_rt[(bp_rt['stage']==stage) & (bp_rt['band']==band) & (bp_rt['region']==region) & (bp_rt['session']==session) & (bp_rt['group_id']==group_id)]['reaction time'].mean()
                    new_row = pd.DataFrame({
                        'stage': [stage],
                        'band': [band],
                        'region': [region],
                        'band power': [bp_mean],
                        'reaction time': [rt_mean]
                    })
                    bp_rt_session = pd.concat([bp_rt_session, new_row], ignore_index=True)
bp_rt_session
# 5m23.3s

,stage,band,region,band power,reaction time
0,fixation,theta,frontal,5.645773e-09,0.349690
1,fixation,theta,frontal,2.260586e-09,0.266151
2,fixation,theta,frontal,6.093726e-09,0.323042
3,fixation,theta,frontal,9.643799e-09,0.290066
4,fixation,theta,frontal,6.773277e-08,0.276883
...,...,...,...,...,...
4315,until response,gamma,all,4.570096e-09,0.304395
4316,until response,gamma,all,2.816022e-09,0.256387
4317,until response,gamma,all,2.327699e-09,0.318608
4318,until response,gamma,all,1.640526e-09,0.354130


In [67]:
ccp_table = pd.DataFrame(columns=['stage', 'band', 'region', 'cc', 'p'])
for stage in stage_names:
    for band in freq_band_names:
        for region in region_names:
            bps = bp_rt_session[(bp_rt_session['stage']==stage) & (bp_rt_session['band']==band) & (bp_rt_session['region']==region)]['band power']
            rts = bp_rt_session[(bp_rt_session['stage']==stage) & (bp_rt_session['band']==band) & (bp_rt_session['region']==region)]['reaction time']
            cc, p = stats.pearsonr(bps, rts)
            new_row = pd.DataFrame({
                'stage': [stage],
                'band': [band],
                'region': [region],
                'cc': [cc],
                'p': [p]
            })
            ccp_table = pd.concat([ccp_table, new_row], ignore_index=True)
ccp_table

,stage,band,region,cc,p
0,fixation,theta,frontal,0.208818,0.221625
1,fixation,theta,central,0.193933,0.257074
2,fixation,theta,parietal,0.233340,0.170804
3,fixation,theta,occipital,0.239772,0.158982
4,fixation,theta,temporal,0.312003,0.063955
...,...,...,...,...,...
115,until response,gamma,central,0.180284,0.292717
116,until response,gamma,parietal,0.213574,0.211041
117,until response,gamma,occipital,0.215860,0.206080
118,until response,gamma,temporal,0.343362,0.040338


In [68]:
ccp_table.sort_values(by=['cc'], ascending=False)

,stage,band,region,cc,p
112,until response,beta,temporal,0.400019,0.015636
13,fixation,beta,central,0.379614,0.022399
88,response,beta,temporal,0.375607,0.023978
16,fixation,beta,temporal,0.369132,0.026724
70,wait,gamma,temporal,0.363446,0.029344
...,...,...,...,...,...
7,fixation,alpha,central,0.083996,0.626223
55,wait,alpha,central,0.067210,0.696930
31,cue,alpha,central,0.047309,0.784092
43,cue,gamma,central,0.039571,0.818762
